### Estudo de Correlação entre Temperatura e Cobertura Vegetal

Este Jupyter Notebook faz parte do módulo de estudo de correlação do projeto PRECORSIA.  
Ele se concentra na análise da correlação entre a temperatura e a cobertura vegetal.

#### Como Utilizar Este Notebook

1. Faça uma cópia deste notebook para o seu ambiente de desenvolvimento.

2. Certifique-se de que você tem todas as dependências necessárias instaladas. Você pode fazê-lo executando o seguinte comando em seu ambiente:

        `pip install numpy matplotlib earthengine-api geemap Pillow --upgrade`

3. Execute cada célula do notebook sequencialmente para realizar a análise. Certifique-se de ler e entender cada passo, e ajuste o código, conforme necessário, para atender aos seus requisitos específicos.

4. Se você desejar contribuir com melhorias ou adicionar novas análises, siga as diretrizes no README do sub-módulo para contribuições.  
Você pode criar um novo notebook ou fazer modificações neste, se necessário.

#### Sobre a autorização para utilizar os serviços Google Earth Engine

Sempre que inicializar a aplicação, será solicitado acesso ao serviço GEE através da sua conta google pessoal.  
Para prover acesso e utilizar a API, relize a autorização executando o código abaixo:
```python
import ee
ee.Authenticate()
ee.Initialize()
```
Você será direcionado(a) à uma página WEB, onde será requisitado permissões de acesso.  

> Granting permission. This creates a web application definition controlled by your project provided above. After you click Generate Token, Google will ask for your permission to grant the application access to your data. See details in the step-by-step guide.

> Expiry period. The granted permissions will expire in a week, after which you'll need to call ee.Authenticate() again.

> Revoking permissions. You can view all applications connected to your account, and revoke permissions if needed, on https://myaccount.google.com/permissions. Search for "Earth Engine Notebook Client" to find the application defined by this page.

> Technical details. The web application is defined by a development-mode "OAuth2 Client" on your specified project, which you can manage on the Google Cloud Console.

In [1]:
# Importing GEE dependencies
from helpers.geeApi import PrecorsiaGee
import ee

# Importing analytical tools
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os

#### Image availability list

In [2]:
dwv1 = PrecorsiaGee('GOOGLE/DYNAMICWORLD/V1', [0.00788292, 0.22374485])
lstv3 = PrecorsiaGee('JAXA/GCOM-C/L3/LAND/LST/V3', [10900, 16591])

In [3]:
# Execute only once per session
PrecorsiaGee.init()

In [4]:
START = ee.Date('2023-01-01')
END = START.advance(90, 'day')
geolocation = [16.0569, 51.6608]

In [5]:
dwv1_list = dwv1.list(geolocation, [START, END])
lstv3_list = lstv3.list(geolocation, [START, END])

dwv1_list, lstv3_list = PrecorsiaGee.correlate_dates(dwv1_list, lstv3_list, 8)

#### Image requests and data acquisition

commit to memory

In [6]:
for id in dwv1_list:
    dwv1_image = dwv1.image(id['id'], ['grass'], geolocation)
    plt.imsave(f'./buffer/%s.png' % id['id'], dwv1_image, cmap='gray')

In [7]:
for id in lstv3_list:
    lstv3_image = lstv3.image(id['id'], ['LST_AVE'], geolocation)
    plt.imsave(f'./buffer/%s.png' % id['id'], lstv3_image, cmap='gray')

filter using zero counting

In [8]:
# Fix this by deleting the image's date pair

for list in [dwv1_list, lstv3_list]:

    # Step one, calcula a quantidades de pixel zeros
    _zeros_class = []
    for id in [image['id'] for image in list]:
        _img = Image.open(f'./buffer/%s.png' % id).convert('L')
        _img_arr = np.array(_img)
        _zeros = np.count_nonzero(_img_arr == 0)
        _zeros_class.append({'id': id, 'zeros': (_zeros / np.prod(_img_arr.shape))})

    # Step two, descarta imagens acima de um threshold
    for image in _zeros_class:
        if image['zeros'] > 0.33:
            os.remove('./buffer/%s.png' % image['id'])
            _zeros_class.remove(image)

    # Step tree, seleciona 3 melhores imagens
    _zeros_class = sorted(_zeros_class, key=lambda z: z['zeros'])
    _best_class = _zeros_class[:3]

    # Step four, imagem padrão com a média das melhores imagens
    _best_image = np.zeros(_img_arr.shape)
    for image in _best_class:
        _img = Image.open(f'./buffer/%s.png' % image['id']).convert('L')
        _img_arr = np.array(_img)
        _best_image += _img_arr * 0.3332

    # Step five, para cada imagem substituir os pixels zero pelos da imagem padrão
    # Step six, save the image back into the file system
    for image in _best_class:
        _img = Image.open(f'./buffer/%s.png' % image['id']).convert('L')
        _img_arr = np.array(_img)

        _img_mask = _img_arr == 0
        _img_filtered = (_best_image * _img_mask) + _img_arr

        plt.imsave(f'./buffer/%s.png' % image['id'], _img_filtered, cmap='gray')


---
### Add the image processing and data extraction

---
### Create the study by processing and visualizing the data